In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/佩玲論文資料/data_詞幹還原.csv",encoding = 'unicode_escape')
df 

,company,sentence,text_wo_stopfreq,text_SnowballStemmer,text_lemmatized
0,Amedisys,the risks described below and risks described ...,risks described risks described elsewhere form...,risk describ risk describ elsewher form 10k co...,risk describ risk describ elsewher form 10k co...
1,Amedisys,the risk factors described below and elsewhere...,risk factors described elsewhere form 10k risk...,risk factor describ elsewher form 10k risk fac...,risk factor describ elsewher form 10k risk fac...
2,Amedisys,our business and consolidated financial condit...,business consolidated financial condition resu...,busi consolid financi condit result oper cash ...,busi consolid financi condit result oper cash ...
3,Amedisys,if any of the following risks are actually rea...,following risks actually realized business con...,follow risk actual realiz busi consolid financ...,follow risk actual realiz busi consolid financ...
4,Amedisys,in that case the trading price of our common s...,case trading price common stock decline,case trade price common stock could declin,case trade price common stock could declin
...,...,...,...,...,...
4735,UnitedHealth,we are also required by law or regulation to m...,also required law regulation maintain specific...,also requir law regul maintain specif prescrib...,also requir law regul maintain specif prescrib...
4736,UnitedHealth,the levels of capitalization required depend p...,levels capitalization required depend primaril...,level capit requir depend primarili volum prem...,level capit requir depend primarili volum prem...
4737,UnitedHealth,in most states we are required to seek approva...,states required seek approval state regulatory...,state requir seek approv state regulatori auth...,state requir seek approv state regulatori auth...
4738,UnitedHealth,an inability of our regulated subsidiaries to ...,inability regulated subsidiaries pay dividends...,inabl regul subsidiari pay dividend parent com...,inabl regul subsidiari pay dividend parent com...


In [ ]:
df.drop(["text_wo_stopfreq", "text_SnowballStemmer"], axis=1, inplace=True)

In [ ]:
df

,company,sentence,text_lemmatized
0,Amedisys,the risks described below and risks described ...,risk describ risk describ elsewher form 10k co...
1,Amedisys,the risk factors described below and elsewhere...,risk factor describ elsewher form 10k risk fac...
2,Amedisys,our business and consolidated financial condit...,busi consolid financi condit result oper cash ...
3,Amedisys,if any of the following risks are actually rea...,follow risk actual realiz busi consolid financ...
4,Amedisys,in that case the trading price of our common s...,case trade price common stock could declin
...,...,...,...
4735,UnitedHealth,we are also required by law or regulation to m...,also requir law regul maintain specif prescrib...
4736,UnitedHealth,the levels of capitalization required depend p...,level capit requir depend primarili volum prem...
4737,UnitedHealth,in most states we are required to seek approva...,state requir seek approv state regulatori auth...
4738,UnitedHealth,an inability of our regulated subsidiaries to ...,inabl regul subsidiari pay dividend parent com...


In [ ]:
df['Financial'] = pd.Series()
df['Strategic'] = pd.Series()
df['Operational'] = pd.Series()
df['Hazard'] = pd.Series()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future versio

In [ ]:
df

,company,sentence,text_lemmatized,Financial,Strategic,Operational,Hazard
0,Amedisys,the risks described below and risks described ...,risk describ risk describ elsewher form 10k co...,NaN,NaN,NaN,NaN
1,Amedisys,the risk factors described below and elsewhere...,risk factor describ elsewher form 10k risk fac...,NaN,NaN,NaN,NaN
2,Amedisys,our business and consolidated financial condit...,busi consolid financi condit result oper cash ...,NaN,NaN,NaN,NaN
3,Amedisys,if any of the following risks are actually rea...,follow risk actual realiz busi consolid financ...,NaN,NaN,NaN,NaN
4,Amedisys,in that case the trading price of our common s...,case trade price common stock could declin,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4735,UnitedHealth,we are also required by law or regulation to m...,also requir law regul maintain specif prescrib...,NaN,NaN,NaN,NaN
4736,UnitedHealth,the levels of capitalization required depend p...,level capit requir depend primarili volum prem...,NaN,NaN,NaN,NaN
4737,UnitedHealth,in most states we are required to seek approva...,state requir seek approv state regulatori auth...,NaN,NaN,NaN,NaN
4738,UnitedHealth,an inability of our regulated subsidiaries to ...,inabl regul subsidiari pay dividend parent com...,NaN,NaN,NaN,NaN


In [ ]:
# df["Financial"] = 0
# df["Strategic"] = 0
# df["Operational"] = 0
# df["Hazard"] = 0

In [ ]:
df

,company,year,sentence,text_lemmatized,Financial,Strategic,Operational,Hazard
0,Allscripts,2018,our business financial condition operating res...,busi financi condit oper result stock price ma...,1,0,0,0
1,Allscripts,2018,any one or more of suchfactors some of which a...,one suchfactor outsid control could directli i...,2,0,0,0
2,Allscripts,2018,because of the following factors as well as ot...,follow factor well factor af ect financi condi...,2,0,0,0
3,Allscripts,2018,these risk factors may be important to underst...,risk factor may import understand statement ma...,0,0,0,0
4,Allscripts,2018,thefollowing information should be read in con...,thefollow inform read conjunct part ii item 7 ...,3,0,0,0
...,...,...,...,...,...,...,...,...
25055,Vocera Communications,2020,the trading market for our common stock depend...,trade market common stock depend part research...,0,0,0,0
25056,Vocera Communications,2020,we do not control these analysts or the conten...,control analyst content opinion includ report,0,0,0,0
25057,Vocera Communications,2020,the price of our common stock could decline if...,price common stock could declin one analyst do...,0,0,0,0
25058,Vocera Communications,2020,if one or more analysts cease coverage of our ...,one analyst ceas coverag compani fail regularl...,1,0,0,0


In [ ]:
#df["Financial"][0] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
len(df.index)

25060

In [ ]:
from collections import Counter

In [ ]:
# #計算出現次數
# my_str = "previous announc june 1 2021 complet acquisit allianc healthcar wba 6 6020 million cash subject certain purchas price adjust 2291 million compani common stock 2 million share compani june 1 2021 open stock price 11454 per share 969 million estim accru consider 61 million equiti con"
# counter = Counter(my_str)
# print(counter['june'])

0


In [ ]:
# counter = Counter(df["text_lemmatized"][643])
# df["Financial"][643] = counter['accru']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df["Financial"][643]

0

In [ ]:
df["text_lemmatized"][643]

'certain comput softwar product regul medic devic feder food drug cosmet act'

In [ ]:
# my_string = "a Mary had a little lamb a bae aaa had2222 had"
# print(len(re.findall("\s*had\s?$", my_string)))
#===成功===
# my_string = "a Mary had a little lamb a bae aaa had2222 had"
# print(len(re.findall("\s*abc\s+", my_string))+len(re.findall("\s*abc\s?$", my_string)))


0


In [ ]:
# my_string = df["text_lemmatized"][643]
# my_string
# int(len(re.findall("\s*accru\s+", my_string))+len(re.findall("\s*accru\s?$", my_string)))

1

In [ ]:
# int(df["Financial"][643])

4

In [ ]:
Fintxt = ["accru", "cumul", "actuari", "asset", "capit", "claim", "conting liabil", "credit" ,"currenc" ,"debt", "earn loss", "econom", "exchang risk", "expect loss", "financi", "gain loss", "hedg",
          "impair", "incom", "infring", "insur", "accrual", "liquid", "loan", "carryforward", "monetari", "net loss", "oper loss", "pension", "potenti liabil", "potenti loss", "rate risk", "agreement",
          "tax", "warrant", "exchang", "loan"]
#lawsuit bankruptci busi interrupt reserv econom uncertainti enterpris inher joint legal polit public safeti reput secur litig ventur
Stratxt = ["lawsuit", "bankruptci", "busi", "interrupt", "reserv", "econom", "uncertainti", "enterpris", "inher", "joint", "legal", "polit", "public safeti", "reput", "secur litig", "ventur"]
#busi disrupt busi risk commerci consum oblig default develop futur claim invest manag oper disrupt patent product profession contract safeti signific damag system failur oper risk
Opertxt = ["busi disrupt", "busi risk", "commerci", "consum", "oblig", "default", "develop", "futur claim", "invest", "manag", "oper disrupt", "patent", "profession", "contract", "safeti", "signific damag",
           "system failur", "oper risk"]
Hazatxt = ["damag aris", "environment", "hazard", "medic", "wast", "secur system"]

In [ ]:
Fintxt

['accru',
 'cumul',
 'actuari',
 'asset',
 'capit',
 'claim',
 'conting liabil',
 'credit',
 'currenc',
 'debt',
 'earn loss',
 'econom',
 'exchang risk',
 'expect loss',
 'financi',
 'gain loss',
 'hedg',
 'impair',
 'incom',
 'infring',
 'insur',
 'accrual',
 'liquid',
 'loan',
 'carryforward',
 'monetari',
 'net loss',
 'oper loss',
 'pension',
 'potenti liabil',
 'potenti loss',
 'rate risk',
 'agreement',
 'tax',
 'warrant',
 'exchang',
 'loan']

In [ ]:
Stratxt

['lawsuit',
 'bankruptci',
 'busi',
 'interrupt',
 'reserv',
 'econom',
 'uncertainti',
 'enterpris',
 'inher',
 'joint',
 'legal',
 'polit',
 'public safeti',
 'reput',
 'secur litig',
 'ventur']

In [ ]:
Opertxt

['busi disrupt',
 'busi risk',
 'commerci',
 'consum',
 'oblig',
 'default',
 'develop',
 'futur claim',
 'invest',
 'manag',
 'oper disrupt',
 'patent',
 'profession',
 'contract',
 'safeti',
 'signific damag',
 'system failur',
 'oper risk']

In [ ]:
len(Hazatxt)

6

In [ ]:
Hazatxt[1]

'environment'

In [ ]:
df["Financial"] = 0
df["Strategic"] = 0
df["Operational"] = 0
df["Hazard"] = 0

In [ ]:
#Financial
for j in range(len(Fintxt)):
  for i in range(len(df.index)):
    my_string = str(df["text_lemmatized"][i])
    df["Financial"][i] = df["Financial"][i] + len(re.findall(Fintxt[j], my_string))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# #Financial
# for j in range(len(Fintxt)):
#   for i in range(len(df.index)):
#     my_string = str(df["text_lemmatized"][i])
#     df["Financial"][i] = df["Financial"][i] + len(re.findall(Fintxt[j], my_string))

In [ ]:
#Stratxt
for j in range(len(Stratxt)):
  for i in range(len(df.index)):
    my_string = str(df["text_lemmatized"][i])
    df["Strategic"][i] = df["Strategic"][i] + len(re.findall(Stratxt[j], my_string))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#Opertxt
for j in range(len(Opertxt)):
  for i in range(len(df.index)):
    my_string = str(df["text_lemmatized"][i])
    df["Operational"][i] = df["Operational"][i] + len(re.findall(Opertxt[j], my_string))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#Hazatxt
for j in range(len(Hazatxt)):
  for i in range(len(df.index)):
    my_string = str(df["text_lemmatized"][i])
    df["Hazard"][i] = df["Hazard"][i] + len(re.findall(Hazatxt[j], my_string))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df

,company,sentence,text_lemmatized,Financial,Strategic,Operational,Hazard
0,Amedisys,the risks described below and risks described ...,risk describ risk describ elsewher form 10k co...,1,1,0,0
1,Amedisys,the risk factors described below and elsewhere...,risk factor describ elsewher form 10k risk fac...,0,0,0,0
2,Amedisys,our business and consolidated financial condit...,busi consolid financi condit result oper cash ...,2,2,0,0
3,Amedisys,if any of the following risks are actually rea...,follow risk actual realiz busi consolid financ...,1,1,0,0
4,Amedisys,in that case the trading price of our common s...,case trade price common stock could declin,0,0,0,0
...,...,...,...,...,...,...,...
4735,UnitedHealth,we are also required by law or regulation to m...,also requir law regul maintain specif prescrib...,1,0,0,0
4736,UnitedHealth,the levels of capitalization required depend p...,level capit requir depend primarili volum prem...,1,0,0,0
4737,UnitedHealth,in most states we are required to seek approva...,state requir seek approv state regulatori auth...,0,0,0,0
4738,UnitedHealth,an inability of our regulated subsidiaries to ...,inabl regul subsidiari pay dividend parent com...,2,2,1,0


In [ ]:
df.to_csv('/content/drive/MyDrive/佩玲論文資料/data_label好的_沒有刪除為0資料.csv', encoding = 'utf-8',index = False)

In [ ]:
# df["Financial"]== 0 and df["Strategic"]== 0
np.any(df["Financial"][0]== 0) and np.any(df["Strategic"][0]== 0) and np.any(df["Operational"][0]== 0) and np.any(df["Hazard"][0]== 0)

False

In [ ]:
df2 = df

In [ ]:
#去除四個欄位風險都為0的資料
for i in range(len(df.index)):
  if np.any(df["Financial"][i] == 0) and np.any(df["Strategic"][i]== 0) and np.any(df["Operational"][i]== 0) and np.any(df["Hazard"][i]== 0) == True:
    df2 = df2.drop(labels=i)
  

In [ ]:
df2

,company,year,sentence,text_lemmatized,Financial,Strategic,Operational,Hazard
0,Allscripts,2018,our business financial condition operating res...,busi financi condit oper result stock price ma...,1,1,0,0
1,Allscripts,2018,any one or more of suchfactors some of which a...,one suchfactor outsid control could directli i...,2,0,0,0
2,Allscripts,2018,because of the following factors as well as ot...,follow factor well factor af ect financi condi...,2,0,1,0
4,Allscripts,2018,thefollowing information should be read in con...,thefollow inform read conjunct part ii item 7 ...,3,0,1,0
8,Allscripts,2018,some of our competitors may be more establishe...,competitor may establish benefit greaternam re...,1,0,0,0
...,...,...,...,...,...,...,...,...
25053,Vocera Communications,2020,regardless of the outcome these matters or fut...,regardless outcom matter futur litig may requi...,1,1,1,0
25054,Vocera Communications,2020,if securities or industry analysts issue an ad...,secur industri analyst issu advers mislead opi...,0,1,0,0
25055,Vocera Communications,2020,the trading market for our common stock depend...,trade market common stock depend part research...,0,1,0,0
25057,Vocera Communications,2020,the price of our common stock could decline if...,price common stock could declin one analyst do...,0,1,0,0


In [ ]:
#重置索引
df2.reset_index(drop=True, inplace=True)

In [ ]:
df2

,company,year,sentence,text_lemmatized,Financial,Strategic,Operational,Hazard
0,Allscripts,2018,our business financial condition operating res...,busi financi condit oper result stock price ma...,1,1,0,0
1,Allscripts,2018,any one or more of suchfactors some of which a...,one suchfactor outsid control could direct ind...,2,0,0,0
2,Allscripts,2018,because of the following factors as well as ot...,follow factor well factor af ect financi condi...,2,0,1,0
3,Allscripts,2018,thefollowing information should be read in con...,thefollow inform read conjunct part ii item 7 ...,3,0,1,0
4,Allscripts,2018,some of our competitors may be more establishe...,competitor may establish benefit greaternam re...,1,0,0,0
...,...,...,...,...,...,...,...,...
22364,UnitedHealth,2020,many of these subsidiaries are regulated by st...,mani subsidiari regul state depart insur simil...,1,0,0,0
22365,UnitedHealth,2020,we are also required by law or regulation to m...,also requir law regul maintain specif prescrib...,1,0,0,0
22366,UnitedHealth,2020,the levels of capitalization required depend p...,level capit requir depend primarili volum prem...,1,0,0,0
22367,UnitedHealth,2020,an inability of our regulated subsidiaries to ...,inabl regul subsidiari pay dividend parent com...,2,2,1,0


In [ ]:
df

,company,year,sentence,text_lemmatized,Financial,Strategic,Operational,Hazard
0,AmerisourceBergen,2018,the following discussion describes certain ris...,follow discus describ certain risk factor beli...,0,1,0,0
1,AmerisourceBergen,2018,these risk factors are in addition to those se...,risk factor addit set forth elsewher report,0,0,0,0
2,AmerisourceBergen,2018,our business operations could also be affected...,busi oper also affect addit factor present kno...,0,1,0,0
3,AmerisourceBergen,2018,the reader should not consider this list to be...,reader consid list complet statement risk unce...,0,1,0,0
4,AmerisourceBergen,2018,our results of operations could be adversely i...,result oper advers impact manufactur price chang,0,0,0,0
...,...,...,...,...,...,...,...,...
9568,UnitedHealth,2020,we are also required by law or regulation to m...,also requir law regul maintain specif prescrib...,1,0,0,0
9569,UnitedHealth,2020,the levels of capitalization required depend p...,level capit requir depend primarili volum prem...,1,0,0,0
9570,UnitedHealth,2020,in most states we are required to seek approv...,state requir seek approv state regulatori auth...,0,0,0,0
9571,UnitedHealth,2020,an inability of our regulated subsidiaries to ...,inabl regul subsidiari pay dividend parent com...,2,2,1,0


In [ ]:
df2.to_csv('/content/drive/MyDrive/佩玲論文資料/alldata_去除風險為0.csv', encoding = 'utf-8',index = False)